# Comparación de perplexity sobre textos de Shakespeare 

Este notebook evalúa el efecto del fine-tuning estilístico mediante perplexity.

Se compara la perplexity promedio de:

- Mistral 7B base

- Mistral 7B fine-tuneado con LoRA

sobre dos tipos de texto:

- frases de Shakespeare (separadas de antemano del corpus con el que se hizo el finetune)

- frases de un corpus no relacionado (inglés moderno/general)

La hipótesis es que el modelo fine-tuneado reduce su perplexity sobre textos estilo Shakespeare,
a costa de aumentar la perplexity sobre textos fuera de dominio.

## 0. Setup (Colab)

In [ ]:
!pip install -q -U torch transformers accelerate bitsandbytes peft

## 1. Imports.

In [ ]:
import torch
import random
import re
import gc
import math
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

## 2. Rutas y parámetros.

In [ ]:
PATH_TXT_SHAKESPEARE = "/content/drive/MyDrive/StoryWriter/Data/RAW/obras_shakespeare.txt"
PATH_TXT_RANDOM      = "/content/drive/MyDrive/StoryWriter/Data/RAW/easy/general_english_corpus.txt"

PATH_MODELO_BASE     = "/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-7b-instruct-v0.3"
PATH_LORA_ADAPTER    = "/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-finetuneado(lora)"

N_SAMPLES = 50
MIN_CHARS = 50

random.seed(42)

## 3. Helpers.

In [ ]:
def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = re.sub(r'\n+', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def obtener_frases_random(ruta_archivo, num_muestras=N_SAMPLES, min_chars=MIN_CHARS):
    """
    Extrae frases al azar de un archivo de texto.
    Aproximamos frases usando separación por puntos.
    """
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()

    frases = contenido.split('.')
    frases_validas = [
        limpiar_texto(f) for f in frases if len(f) > min_chars
    ]

    if len(frases_validas) <= num_muestras:
        return frases_validas

    return random.sample(frases_validas, num_muestras)

## 4. Muestreo de frases.

In [ ]:
print("Muestreando frases...")

muestras_shakespeare = obtener_frases_random(PATH_TXT_SHAKESPEARE)
muestras_random      = obtener_frases_random(PATH_TXT_RANDOM)

print(f"Shakespeare: {len(muestras_shakespeare)} frases")
print(f"No Shakespeare: {len(muestras_random)} frases")

print("\nEjemplo Shakespeare:")
print(muestras_shakespeare[0][:120], "...")

print("\nEjemplo No Shakespeare:")
print(muestras_random[0][:120], "...")

## 5. Funciones para perplexity y evaluación.

In [ ]:
def calcular_perplexity(model, tokenizer, frases):
    model.eval()
    total_loss = 0
    count = 0

    with torch.no_grad():
        for frase in frases:
            inputs = tokenizer(frase, return_tensors="pt")
            if torch.cuda.is_available():
                inputs = inputs.to("cuda")

            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss

            if not math.isnan(loss.item()):
                total_loss += loss.item()
                count += 1

    if count == 0:
        return float("inf")

    avg_loss = total_loss / count
    return math.exp(avg_loss)

def evaluar_modelo(nombre, ruta_base, ruta_lora=None):
    """
    Evalúa un modelo base o un modelo base + adaptador LoRA.
    """
    print("\n============================================")
    print(f"MODELO: {nombre}")
    print("============================================")

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(ruta_base, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Modelo base
    base_model = AutoModelForCausalLM.from_pretrained(
        ruta_base,
        load_in_4bit=True,
        device_map="auto",
        torch_dtype=torch.float16
    )

    # Adaptador LoRA (si corresponde)
    if ruta_lora is not None:
        model = PeftModel.from_pretrained(base_model, ruta_lora)
    else:
        model = base_model

    print("→ Midiendo perplexity en Shakespeare...")
    ppl_shak = calcular_perplexity(model, tokenizer, muestras_shakespeare)

    print("→ Midiendo perplexity en No-Shakespeare...")
    ppl_rand = calcular_perplexity(model, tokenizer, muestras_random)

    print("\nRESULTADOS:")
    print(f"Perplexity Shakespeare     : {ppl_shak:.4f}")
    print(f"Perplexity No Shakespeare  : {ppl_rand:.4f}")

    # Limpieza de memoria
    del model
    del tokenizer
    del base_model
    torch.cuda.empty_cache()
    gc.collect()

## 7. Ejecución.

In [ ]:
evaluar_modelo(
    nombre="MISTRAL BASE",
    ruta_base=PATH_MODELO_BASE
)

evaluar_modelo(
    nombre="MISTRAL FINE-TUNED (LoRA)",
    ruta_base=PATH_MODELO_BASE,
    ruta_lora=PATH_LORA_ADAPTER
)